In [1]:
import numpy as np
import pylidc as pl
from sklearn.model_selection import train_test_split
from unet import UNet

Using TensorFlow backend.


In [2]:
scans = pl.query(pl.Scan)
n = scans.count()

In [3]:
image_size = (256, 256)

In [5]:
def union_bboxes(anns):
    return tuple((min(d.start), max(d.stop)) for d in zip(*(ann.bbox() for ann in anns)))

In [ ]:
def get_data(i, samples):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    X, y = [], []
    for nodule in scan.cluster_annotations():
        coords = union_bboxes(nodule)
        x_coord, y_coord, z_coord = coords
        y_big = np.zeros((stop - start for start, stop in coords))
        for ann in nodule:
            b = ann.bbox()
            bb = tuple(slice(bi.start - coord[0], bi.stop - coord[0], None) for bi, coord in zip(b, coords))
            y_big[bb] = np.logical_or(y_big[bb], ann.boolean_mask())
        x_mean = int(sum(x_coord) / 2)
        y_mean = int(sum(y_coord) / 2)
        for zi in range(*z_coord):
            for _ in samples:
                x_start = np.random.randint(max(0, x_mean - image_size[0]), min(512 - image_size[0], x_mean))
                y_start = np.random.randint(max(0, y_mean - image_size[1]), min(512 - image_size[1], y_mean))
                X.append(images[zi].pixel_array[x_start:x_start + image_size[0], y_start:y_start + image_size[1]])
                y.append(y_big[x_start:x_start + image_size[0], y_start:y_start + image_size[1], zi - zcoord[0]])

In [ ]:
def get_data(i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([resize(im.pixel_array, image_size, mode='constant') for im in images])
    X = np.expand_dims(X, axis=-1)
    y = np.zeros((512, 512, n))
    for ann in scan.annotations:
        b = ann.bbox()
        y[b] = np.logical_or(y[b], ann.boolean_mask())
    y = np.rollaxis(y, 2, 0)
    y = resize(y, (n, *image_size), mode='constant')
    if only_nodules:
        indices = [l.sum() > 0 for l in y]
        X = X[indices]
        y = y[indices]
    y = np.expand_dims(y, axis=-1)
    return X, y